### DESCRIPTION:
    This example shows how to retrieve data from Azure SQL DB by using Open AI GPT.  
    Asking questions in plain english that gets "translated" by GPT into SQL.
    Using Langchain SQLDatabaseChain 
### REQUIREMENTS:
    1. Create an Azure SQL DB and populate it with data.
    2. Create an .env file in the root folder with the following variables:
      SQL_SERVER="<server>"
      SQL_USER="<user>"
      SQL_PWD="<pwd>"
      SQL_DBNAME="<dbname>"

### Sample questions you can ask:
      List the tables in the database
      How many products are in the Adventure Works database?
      How many Products are color black?
      How many SalesOrderDetail are for the Product AWC Logo Cap ?
      List the top 10 most expensive products
      What are the top 10 highest grossing products in the Adventure Works database?

### For more information about Langchain agent toolkits, see:
  https://github.com/hwchase17/langchain/tree/master/langchain/agents/agent_toolkits


In [80]:
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from dotenv import load_dotenv
import openai
import os
import pymssql

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION") 

SQL_SERVER = os.getenv("SQL_SERVER")
SQL_USER = os.getenv("SQL_USER")
SQL_PWD = os.getenv("SQL_PWD")
SQL_DBNAME = os.getenv("SQL_DBNAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY


In [81]:
def init_llm(model=OPENAI_MODEL_NAME,
             deployment_name=OPENAI_DEPLOYMENT_NAME,
             openai_api_version=OPENAI_DEPLOYMENT_VERSION,
             temperature=0,
             max_tokens=400,
             top_p=1,
             ):
    llm = AzureChatOpenAI( 
                      azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                      deployment_name=deployment_name,
                      model=model,
                      openai_api_version=openai_api_version,
                      temperature=temperature,
                      max_tokens=max_tokens,
                      top_p=top_p
                      )
    return llm

### **Approach 1 - Generate SQL and then run in DB**

In [82]:
# initialize azure OpenAI
llm = init_llm()

from langchain import SQLDatabase

sqlconn = f"mssql+pymssql://{SQL_USER}:{SQL_PWD}@{SQL_SERVER}:1433/{SQL_DBNAME}"
db = SQLDatabase.from_uri(sqlconn)
# chain = create_sql_query_chain(AzureChatOpenAI(temperature=0, api_version="2023-05-15", azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT, deployment_name="gpt-35-turbo", verbose=True), db)

chain = create_sql_query_chain(llm, db)
# generate SQL from question in English


WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


In [83]:
query = chain.invoke({"question":"How many orders do we have?"})
print("Query generated by OpenAI: " + query)

Query generated by OpenAI: SELECT COUNT(SalesOrderID) AS TotalOrders
FROM SalesOrderHeader


In [84]:
# generate SQL from question in English
query = chain.invoke({"question":"List the 3 customers who made the most expensive orders ordered by the total amount in a descending order"})
print("Query generated by OpenAI: " + query)

Query generated by OpenAI: SELECT TOP 3 c.FirstName, c.LastName, soh.TotalDue
FROM Customer c
JOIN SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
ORDER BY soh.TotalDue DESC;


In [85]:
#run generated SQL in DB
db.run(query)

"[('Terry', 'Eminhizer', Decimal('119960.8240')), ('Krishna', 'Sunkammurali', Decimal('108597.9536')), ('Christopher', 'Beck', Decimal('98138.2131'))]"

### **Approach 2 - Use experimental SQL chain**

In [86]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
db_chain.run("How many customers are in the database?")



> Entering new SQLDatabaseChain chain...
How many customers are in the database?
SQLQuery:SELECT COUNT(CustomerID) AS TotalCustomers
FROM Customer
SQLResult: [(847,)]
Answer:There are 847 customers in the database.
> Finished chain.


'There are 847 customers in the database.'

In [87]:
db_chain.run("How many customers have placed orders in year 2008?")



> Entering new SQLDatabaseChain chain...
How many customers have placed orders in year 2008?
SQLQuery:SELECT COUNT(DISTINCT soh.CustomerID) AS NumCustomers
FROM SalesOrderHeader soh
WHERE YEAR(soh.OrderDate) = 2008
SQLResult: [(32,)]
Answer:32 customers have placed orders in the year 2008.
> Finished chain.


'32 customers have placed orders in the year 2008.'

In [88]:
db_chain.run("List the number of customers with more than one address")



> Entering new SQLDatabaseChain chain...
List the number of customers with more than one address
SQLQuery:SELECT COUNT(CustomerID) AS NumCustomers
FROM (
    SELECT CustomerID, COUNT(AddressID) AS NumAddresses
    FROM CustomerAddress
    GROUP BY CustomerID
    HAVING COUNT(AddressID) > 1
) AS Subquery
SQLResult: [(10,)]
Answer:There are 10 customers with more than one address.
> Finished chain.


'There are 10 customers with more than one address.'

In [89]:
db_chain.run("What are the top 10 highest grossing products in the database?")



> Entering new SQLDatabaseChain chain...
What are the top 10 highest grossing products in the database?
SQLQuery:SELECT TOP 10 
    [ProductID],
    SUM([LineTotal]) AS TotalSales
FROM 
    [SalesOrderDetail]
GROUP BY 
    [ProductID]
ORDER BY 
    TotalSales DESC;
SQLResult: [(969, Decimal('37191.492000')), (783, Decimal('37178.838000')), (976, Decimal('36486.235500')), (782, Decimal('35801.844000')), (957, Decimal('23413.474656')), (967, Decimal('22887.072000')), (780, Decimal('20879.910000')), (973, Decimal('20411.880000')), (784, Decimal('19277.916000')), (974, Decimal('18692.519308'))]
Answer:The top 10 highest grossing products in the database are as follows:
1. ProductID: 969, TotalSales: $37191.49
2. ProductID: 783, TotalSales: $37178.84
3. ProductID: 976, TotalSales: $36486.24
4. ProductID: 782, TotalSales: $35801.84
5. ProductID: 957, TotalSales: $23413.47
6. ProductID: 967, TotalSales: $22887.07
7. ProductID: 780, TotalSales: $20879.91
8. ProductID: 973, TotalSales: $20411

'The top 10 highest grossing products in the database are as follows:\n1. ProductID: 969, TotalSales: $37191.49\n2. ProductID: 783, TotalSales: $37178.84\n3. ProductID: 976, TotalSales: $36486.24\n4. ProductID: 782, TotalSales: $35801.84\n5. ProductID: 957, TotalSales: $23413.47\n6. ProductID: 967, TotalSales: $22887.07\n7. ProductID: 780, TotalSales: $20879.91\n8. ProductID: 973, TotalSales: $20411.88\n9. ProductID: 784, TotalSales: $19277.92\n10. ProductID: 974, TotalSales: $18692.52'

#### Use prompts to generate a question and avoid chatty answers

In [91]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:


Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Do not add any additional text to the SQLResult.
Only use the following tables:


{table_info}


Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)
new_db_chain = SQLDatabaseChain(llm=llm, database=db, prompt=PROMPT, verbose=False)


/Users/liorking/Projects/AOAI-workshop/.venv/lib/python3.9/site-packages/langchain_experimental/sql/base.py:76: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [92]:
new_db_chain.run(dict(query="Sum up the total revenue", table_info=db.get_table_info(), dialect="ms sql", verbose=False, top_k=10))

'Total Revenue: 708690.153058'